In [1]:
import sys
import os
import cv2
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import random
import matplotlib.pyplot as plt
from numpy import pi, matmul,sqrt,dot,array,zeros,cos,sin,pi,arccos
from torch.utils.data import DataLoader
from func import OR,heatplot,misorientation,mat2plot, match,imgshow,calpoint,L2,L1,ipfread
from Class import Exp,Data,Cluster
import torch.nn.functional as F
import matplotlib.patches as patches

class Dataset(torch.utils.data.Dataset):
    def __init__(self,bef,im,target):
        self.bef=bef
        self.im=im
        self.target=target
    def __getitem__(self,index):
        X=self.bef[index]
        Y=self.target[index]
        return torch.tensor(X,dtype=torch.float32),torch.tensor(Y,dtype=torch.float32)
    def __len__(self):
        return len(self.bef)
def negsample(corner):
    mat=np.zeros([500,500])
    for (i,j) in corner:
        for k in range(max(0,i-49),min(500,i+50)):
            for l in range(max(0,j-49),min(500,j+50)):
                mat[k,l]=1
    cand = []
    for i in range(451):
        for j in range(451):
            if not mat[i][j]:
                cand.append([i,j])
    return random.sample(cand,len(corner))


In [2]:
def create_dataset(root="data/train/",feature=["orient","MAD"]):
    corner=torch.load("data/corner.pkl")
    bef=[]
    im=[]
    target=[]
    h=w=50
    for date in os.listdir(root):
        path=root+date+"/before/"
        data=Data(path)
        data.data["orient"]=data.data["orient"].reshape(data.h,data.w,-1)
        for (i,j) in corner[date]:
            im.append(ipfread(path)[i:i+h,j:j+w]/255.)
            app=[]
            for ele in feature:
                if ele=="orient":
                    app.append(data.data[ele][i:i+h,j:j+w])
                else:
                    app.append(data.data[ele][i:i+h,j:j+w,np.newaxis])
            bef.append(np.concatenate(app,axis=2))
            target.append(1)
        for (i,j) in negsample(corner[date]):
            im.append(ipfread(path)[i:i+h,j:j+w]/255.)
            app=[]
            for ele in feature:
                if ele=="orient":
                    app.append(data.data[ele][i:i+h,j:j+w])
                else:
                    app.append(data.data[ele][i:i+h,j:j+w,np.newaxis])
            bef.append(np.concatenate(app,axis=2))
            target.append(0)
        
    bef=np.transpose(np.array(bef),(0,3,1,2))
    im=np.array(im)
    target=np.array(target)[:,np.newaxis]
    return Dataset(bef,im,target)
train=create_dataset("data/train/")
test=create_dataset("data/test/")

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.relu = nn.ReLU() # activation
        self.maxpool = nn.MaxPool2d(kernel_size=2) #output_shape=(16,24,24)
        self.cnn1 = nn.Conv2d(in_channels=10, out_channels=12, kernel_size=3, stride=1, padding=0) #output_shape=(16,48,48)
        self.cnn2 = nn.Conv2d(in_channels=12, out_channels=18, kernel_size=3, stride=1, padding=1) #output_shape=(32,22,22
        self.cnn3 = nn.Conv2d(in_channels=18, out_channels=24, kernel_size=3, stride=1, padding=0) #output_shape=(32,22,22
        self.fc1 = nn.Linear(24 * 5 * 5, 1) 
        self.sig = nn.Sigmoid()
    def forward(self, x):
        # Convolution 1 50
        out = self.cnn1(x) # 48
        out = self.relu(out)
        out = self.maxpool(out) #24
        out = self.cnn2(out) #24
        out = self.relu(out) 
        out = self.maxpool(out) #12
        out = self.cnn3(out) #10
        out = self.relu(out)
        out = self.maxpool(out) #5
        out = out.view(out.size(0), -1)
        # Linear function (readout)
        out = self.fc1(out)
        out = self.sig(out)
        return out

In [4]:
train_loader= DataLoader(train, batch_size=64, shuffle=True,  num_workers=0, drop_last=True )
test_loader= DataLoader(test, batch_size=64, shuffle=True,  num_workers=0,  drop_last=False )

In [ ]:
model = CNN()
Loss = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
max_acc = 0
epoch = 1000
for ep in range(epoch):
    for batch_ndx, sample in enumerate(train_loader):
        optimizer.zero_grad()
        X,Y= sample
        output=model(X)
        loss = Loss(output,Y)
        loss.backward()
        optimizer.step()
#         print("train ",ep,loss.item())
    model.eval()
    cum=0
    loss=0
    for batch_ndx, sample in enumerate(test_loader):
        X,Y= sample
        output=model(X)
        loss+=Loss(output,Y)
        cum+= np.sum((output.detach().cpu().numpy()>0.5)  == Y.detach().cpu().numpy().astype("bool"))
    acc=cum/len(test)
    if acc>max_acc:
        max_acc=acc
        min_loss = loss.item()
        print(ep,min_loss)
        print("acc= ",cum,"/",len(test))
        torch.save(model,"model/%.3f"%(acc))
        
    

0 1.3877317905426025
acc=  50 / 102
1 1.3850594758987427
acc=  51 / 102


/home/kisaki/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


2 1.3858587741851807
acc=  54 / 102
3 1.384732723236084
acc=  56 / 102
5 1.3830680847167969
acc=  59 / 102
6 1.383279800415039
acc=  60 / 102
53 1.321670651435852
acc=  63 / 102
55 1.3161487579345703
acc=  64 / 102
67 1.2870895862579346
acc=  65 / 102
68 1.2844429016113281
acc=  66 / 102
69 1.2884199619293213
acc=  68 / 102
74 1.2776306867599487
acc=  69 / 102
75 1.2469499111175537
acc=  70 / 102
76 1.2388029098510742
acc=  71 / 102
78 1.234377384185791
acc=  72 / 102
85 1.187355399131775
acc=  73 / 102
86 1.1768972873687744
acc=  74 / 102
88 1.1630933284759521
acc=  75 / 102
157 1.0142565965652466
acc=  76 / 102
169 1.083349585533142
acc=  77 / 102
